In [1]:
spark

Intitializing Scala interpreter ...

Spark Web UI available at http://172.29.64.245:4040
SparkContext available as 'sc' (version = 3.3.2, master = local[*], app id = local-1682226123927)
SparkSession available as 'spark'


res0: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@5450c6e8


In [4]:
//구메 이력 데이터를 사용해 파티션을 훨씬 적은 수로 분할할 수 있도록 리파티셔닝하고 빠르게 접근할 수 있도록 캐싱
//파티션 수를 줄이는 이유 : 적은 양의 데이터를 가진 수많은 파일이 존재하기 때문
val df = spark.read.format("csv")
.option("header", "true")
.option("inferSchema", "true")
.load("sample_data/retail-data/all/*.csv")
.coalesce(5)

df.cache()
df.createOrReplaceTempView("dfTable")

23/04/23 14:05:20 WARN CacheManager: Asked to cache already cached data.


df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [InvoiceNo: string, StockCode: string ... 6 more fields]


In [6]:
//데이터셋의 전체 크기를 알아보는 용도 & 메모리에 DataFrame 캐싱 작업을 수행하는 용도
df.count() == 541909

res5: Boolean = true


# 7.1 집계 함수
- 모든 집계는 특별한 경우를 제외한다면 함수를 사용
- 집계 함수는 org.apache.spark.sql.functions 패키지에서 찾아볼 수 있음

## 7.1.1 count
- count 함수는 두 가지 방식으로 사용할 수 있음
    1. count 함수에 특정 컬럼을 지정하는 방식
    2. count(\*)나 count count(1)을 사용하는 방식

In [7]:
import org.apache.spark.sql.functions.count

df.select(count("StockCode")).show()

+----------------+
|count(StockCode)|
+----------------+
|          541909|
+----------------+



import org.apache.spark.sql.functions.count


In [8]:
spark.sql("""
SELECT COUNT(*) FROM dfTable
""").show()

+--------+
|count(1)|
+--------+
|  541909|
+--------+



## 7.1.2 countDistinct
- 전체 레코드 수가 아닌 고유 레코드 수

In [9]:
import org.apache.spark.sql.functions.countDistinct

df.select(countDistinct("StockCode")).show()

+-------------------------+
|count(DISTINCT StockCode)|
+-------------------------+
|                     4070|
+-------------------------+



import org.apache.spark.sql.functions.countDistinct


## 7.1.3 approx_count_distinct
- 대규모 데이터셋을 다루다 보면 정확한 고유 개수가 무의미할 때도 있음
- 어느 정도 수준의 정확도를 가지는 근사치만으로도 유의미하다면 해당 함수를 통해 근사치 계산

In [10]:
import org.apache.spark.sql.functions.approx_count_distinct

df.select(approx_count_distinct("StockCode", 0.1)).show()

+--------------------------------+
|approx_count_distinct(StockCode)|
+--------------------------------+
|                            3364|
+--------------------------------+



import org.apache.spark.sql.functions.approx_count_distinct


## 7.1.4 first와 last
- DataFrame의 첫 번째 값이나 마지막 값을 얻을 때 사용

In [11]:
import org.apache.spark.sql.functions.{first, last}

df.select(first("StockCode"), last("StockCode")).show()

+----------------+---------------+
|first(StockCode)|last(StockCode)|
+----------------+---------------+
|          85123A|          22138|
+----------------+---------------+



import org.apache.spark.sql.functions.{first, last}


## 7.1.5 min과 max

In [12]:
import org.apache.spark.sql.functions.{min, max}

df.select(min("Quantity"), max("Quantity")).show()

+-------------+-------------+
|min(Quantity)|max(Quantity)|
+-------------+-------------+
|       -80995|        80995|
+-------------+-------------+



import org.apache.spark.sql.functions.{min, max}


## 7.1.6 sum

In [13]:
import org.apache.spark.sql.functions.sum

df.select(sum("Quantity")).show()

+-------------+
|sum(Quantity)|
+-------------+
|      5176450|
+-------------+



import org.apache.spark.sql.functions.sum


## 7.1.7 sumDistinct
- 고윳값 합산

In [14]:
import org.apache.spark.sql.functions.sumDistinct

df.select(sumDistinct("Quantity")).show()

+----------------------+
|sum(DISTINCT Quantity)|
+----------------------+
|                 29310|
+----------------------+



import org.apache.spark.sql.functions.sumDistinct


## 7.1.8 avg
- sum 함수의 결과를 count 함수의 결과로 나누어 평균값을 구할 수 있지만, 스파크의 avg 함수나 mean 함수를 사용하면 평균값을 더 쉽게 구할 수 있음

In [15]:
import org.apache.spark.sql.functions.{sum, count, avg, expr}

df.select(
    count("Quantity").alias("total_transactions"),
    sum("Quantity").alias("total_purchases"),
    avg("Quantity").alias("avg_purchases"),
    expr("mean(Quantity)").alias("mean_purchases"))
.selectExpr(
    "total_purchases/total_transactions",
    "avg_purchases",
    "mean_purchases").show()

+--------------------------------------+----------------+----------------+
|(total_purchases / total_transactions)|   avg_purchases|  mean_purchases|
+--------------------------------------+----------------+----------------+
|                      9.55224954743324|9.55224954743324|9.55224954743324|
+--------------------------------------+----------------+----------------+



import org.apache.spark.sql.functions.{sum, count, avg, expr}


## 7.1.9 분산과 표준편차

- 주변에 데이터가 분포된 정도를 측정하는 방법
- 분산은 평균과의 차이를 제곱한 결과의 평균
- 표준편차는 분산의 제곱근

In [16]:
import org.apache.spark.sql.functions.{var_pop, stddev_pop}
import org.apache.spark.sql.functions.{var_samp, stddev_samp}

df.select(var_pop("Quantity"), var_samp("Quantity"),
          stddev_pop("Quantity"), stddev_samp("Quantity")).show()

+-----------------+------------------+--------------------+---------------------+
|var_pop(Quantity)|var_samp(Quantity)|stddev_pop(Quantity)|stddev_samp(Quantity)|
+-----------------+------------------+--------------------+---------------------+
|47559.30364660918| 47559.39140929887|  218.08095663447824|   218.08115785023443|
+-----------------+------------------+--------------------+---------------------+



import org.apache.spark.sql.functions.{var_pop, stddev_pop}
import org.apache.spark.sql.functions.{var_samp, stddev_samp}


## 7.1.10 비대칭도와 첨도
- 비대칭도와 첨도 모두 데이터의 변곡점을 측정하는 방법
- 비대칭도는 데이터 평균의 비대칭 정도를 측정, 첨도는 데이터 끝 부분을 측정

In [17]:
import org.apache.spark.sql.functions.{skewness, kurtosis}

df.select(skewness("Quantity"), kurtosis("Quantity")).show()

+-------------------+------------------+
| skewness(Quantity)|kurtosis(Quantity)|
+-------------------+------------------+
|-0.2640755761052702|119768.05495532964|
+-------------------+------------------+



import org.apache.spark.sql.functions.{skewness, kurtosis}


## 7.1.11 공분산과 상관관계
- 단일컬럼이 아닌 두 컬럼값 사이의 영향도를 비교
- cov, corr 함수를 사용해 공분산과 상관관계를 계산

In [18]:
import org.apache.spark.sql.functions.{corr, covar_pop, covar_samp}

df.select(corr("InvoiceNo", "Quantity"), covar_samp("InvoiceNo", "Quantity"),
          covar_pop("InvoiceNo", "Quantity")).show()

+-------------------------+-------------------------------+------------------------------+
|corr(InvoiceNo, Quantity)|covar_samp(InvoiceNo, Quantity)|covar_pop(InvoiceNo, Quantity)|
+-------------------------+-------------------------------+------------------------------+
|     4.912186085641572E-4|              1052.728054391933|            1052.7260778758289|
+-------------------------+-------------------------------+------------------------------+



import org.apache.spark.sql.functions.{corr, covar_pop, covar_samp}


## 7.1.12 복합 데이터 타입의 집계

In [19]:
import org.apache.spark.sql.functions.{collect_set, collect_list}

df.agg(collect_set("Country"), collect_list("Country")).show()

+--------------------+---------------------+
|collect_set(Country)|collect_list(Country)|
+--------------------+---------------------+
|[Portugal, Italy,...| [United Kingdom, ...|
+--------------------+---------------------+



import org.apache.spark.sql.functions.{collect_set, collect_list}


# 7.2 그룹화
- 단일 컬럼의 데이터를 그룹화하고 해당 그룹의 다른 여러 컬럼을 사용해서 계산하기 위해 카테고리형 데이터를 사용

In [20]:
//송장번호(InvoiceNo)를 기준으로 그룹을 만들고 그룹별 물품 수를 카운트
df.groupBy("InvoiceNo", "CustomerId").count().show()

+---------+----------+-----+
|InvoiceNo|CustomerId|count|
+---------+----------+-----+
|   536846|     14573|   76|
|   537026|     12395|   12|
|   537883|     14437|    5|
|   538068|     17978|   12|
|   538279|     14952|    7|
|   538800|     16458|   10|
|   538942|     17346|   12|
|  C539947|     13854|    1|
|   540096|     13253|   16|
|   540530|     14755|   27|
|   541225|     14099|   19|
|   541978|     13551|    4|
|   542093|     17677|   16|
|   543188|     12567|   63|
|   543590|     17377|   19|
|  C543757|     13115|    1|
|  C544318|     12989|    1|
|   544578|     12365|    1|
|   545165|     16339|   20|
|   545289|     14732|   30|
+---------+----------+-----+
only showing top 20 rows



## 7.2.1 표현식을 이용한 그룹화

In [21]:
import org.apache.spark.sql.functions.count

df.groupBy("InvoiceNo").agg(
    count("Quantity").alias("quan"),
    expr("count(Quantity)"))
.show()

+---------+----+---------------+
|InvoiceNo|quan|count(Quantity)|
+---------+----+---------------+
|   536596|   6|              6|
|   536938|  14|             14|
|   537252|   1|              1|
|   537691|  20|             20|
|   538041|   1|              1|
|   538184|  26|             26|
|   538517|  53|             53|
|   538879|  19|             19|
|   539275|   6|              6|
|   539630|  12|             12|
|   540499|  24|             24|
|   540540|  22|             22|
|  C540850|   1|              1|
|   540976|  48|             48|
|   541432|   4|              4|
|   541518| 101|            101|
|   541783|  35|             35|
|   542026|   9|              9|
|   542375|   6|              6|
|  C542604|   8|              8|
+---------+----+---------------+
only showing top 20 rows



import org.apache.spark.sql.functions.count


## 7.2.2 맵을 이용한 그룹화
- 컬럼을 키로, 수행할 집계 함수의 문자열을 값으로 하는 맵 타입을 사용해 트랜스포메이션을 정의할 수 있음

In [22]:
df.groupBy("InvoiceNo").agg("Quantity"->"avg", "Quantity"->"stddev_pop").show()

+---------+------------------+--------------------+
|InvoiceNo|     avg(Quantity)|stddev_pop(Quantity)|
+---------+------------------+--------------------+
|   536596|               1.5|  1.1180339887498947|
|   536938|33.142857142857146|  20.698023172885524|
|   537252|              31.0|                 0.0|
|   537691|              8.15|   5.597097462078001|
|   538041|              30.0|                 0.0|
|   538184|12.076923076923077|   8.142590198943392|
|   538517|3.0377358490566038|  2.3946659604837897|
|   538879|21.157894736842106|  11.811070444356483|
|   539275|              26.0|  12.806248474865697|
|   539630|20.333333333333332|  10.225241100118645|
|   540499|              3.75|  2.6653642652865788|
|   540540|2.1363636363636362|  1.0572457590557278|
|  C540850|              -1.0|                 0.0|
|   540976|10.520833333333334|   6.496760677872902|
|   541432|             12.25|  10.825317547305483|
|   541518| 23.10891089108911|  20.550782784878713|
|   541783|1

# 7.3 윈도우 함수
- 데이터의 특정 '윈도우'를 대상으로 고유의 집계 연산을 수행
- 데이터의 '윈도우'는 현재 데이터에 대한 참조를 사용해 정의
- 윈도우 명세는 함수에 전달될 로우를 결정

- groupBy와의 차이점
    - group-by 함수를 사용하면 모든 로우 레코드가 단일 그룹으로만 이동
    - 윈도우 함수는 프레임에 입력되는 모든 로우에 대해 결괏값을 계산
    - 프레임은 로우 그룹 기반의 테이블을 의미
    - 각 로우는 하나 이상의 프레임에 할당될 수 있음
- 스파크가 지원하는 세 가지 종류의 윈도우 함수
    - 랭크 함수(ranking function)
    - 분석 함수(analytic function)
    - 집계 함수(aggregate function)

In [23]:
import org.apache.spark.sql.functions.{col, to_date}

val dfWithDate = df.withColumn("date", to_date(col("InvoiceDate"), "MM/d/yyyy H:mm"))
dfWithDate.createOrReplaceTempView("dfWithDate")

import org.apache.spark.sql.functions.{col, to_date}
dfWithDate: org.apache.spark.sql.DataFrame = [InvoiceNo: string, StockCode: string ... 7 more fields]


- 윈도우 함수를 정의하기 위해 첫 번째 단계로 윈도우 명세를 만듦
- 여기서 사용하는 partitionBy 메서드는 지금까지 사용해온 파티셔닝 스키마의 개념과는 관련이 없으며 그룹을 어떻게 나눌지 결정하는 것과 유사한 개념
- orderBy 메서드는 파티션의 정렬 방식을 정의
- 그리고 프레임 명세(rowsBetween 구문)는 입력된 로우의 참조를 기반으로 프레임에 로우가 포함될 수 있는지 결정

In [25]:
import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.functions.col

val windowSpec = Window
.partitionBy("CustomerId", "date")
.orderBy(col("Quantity").desc)
.rowsBetween(Window.unboundedPreceding, Window.currentRow)

import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.functions.col
windowSpec: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@14443fe0


- 시간대별 최대 구매 개수를 구하는 예
- 그러려면 위 예제에서 사용한 집계 함수에 컬럼명이나 표현식을 전달해야 함
- 그리고 이 함수를 적용할 데이터 프레임이 정의된 윈도우 명세도 함께 사용

In [27]:
import org.apache.spark.sql.functions.max

val maxPurchaseQuantity = max(col("Quantity")).over(windowSpec)

import org.apache.spark.sql.functions.max
maxPurchaseQuantity: org.apache.spark.sql.Column = max(Quantity) OVER (PARTITION BY CustomerId, date ORDER BY Quantity DESC NULLS LAST ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)


In [28]:
import org.apache.spark.sql.functions.{dense_rank, rank}

val purchaseDenseRank = dense_rank().over(windowSpec)
val purchaseRank = rank().over(windowSpec)

import org.apache.spark.sql.functions.{dense_rank, rank}
purchaseDenseRank: org.apache.spark.sql.Column = DENSE_RANK() OVER (PARTITION BY CustomerId, date ORDER BY Quantity DESC NULLS LAST ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)
purchaseRank: org.apache.spark.sql.Column = RANK() OVER (PARTITION BY CustomerId, date ORDER BY Quantity DESC NULLS LAST ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)


In [33]:
import org.apache.spark.sql.functions.col
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")

dfWithDate.where("CustomerId IS NOT NULL").orderBy("CustomerId")
.select(
    col("CustomerId"),
    col("date"),
    col("Quantity"),
    purchaseRank.alias("quantityRank"),
    purchaseDenseRank.alias("quantityDenseRank"),
    maxPurchaseQuantity.alias("maxPurchaseQuantity"))
.show()

+----------+----------+--------+------------+-----------------+-------------------+
|CustomerId|      date|Quantity|quantityRank|quantityDenseRank|maxPurchaseQuantity|
+----------+----------+--------+------------+-----------------+-------------------+
|     12346|2011-01-18|   74215|           1|                1|              74215|
|     12346|2011-01-18|  -74215|           2|                2|              74215|
|     12347|2010-12-07|      36|           1|                1|                 36|
|     12347|2010-12-07|      30|           2|                2|                 36|
|     12347|2010-12-07|      24|           3|                3|                 36|
|     12347|2010-12-07|      12|           4|                4|                 36|
|     12347|2010-12-07|      12|           4|                4|                 36|
|     12347|2010-12-07|      12|           4|                4|                 36|
|     12347|2010-12-07|      12|           4|                4|             

import org.apache.spark.sql.functions.col


# 7.4 그룹화 셋
- 여러 그룹에 걸쳐 집계

In [38]:
val dfNoNull = dfWithDate.na.drop()

dfNoNull.createOrReplaceTempView("dfNoNull")

dfNoNull: org.apache.spark.sql.DataFrame = [InvoiceNo: string, StockCode: string ... 7 more fields]


In [40]:
spark.sql("""
SELECT CustomerId, stockCode, sum(Quantity) FROM dfNoNull
GROUP BY CustomerId, stockcode
ORDER BY CustomerId DESC, stockCode DESC
""").show()

+----------+---------+-------------+
|CustomerId|stockCode|sum(Quantity)|
+----------+---------+-------------+
|     18287|    85173|           48|
|     18287|   85040A|           48|
|     18287|   85039B|          120|
|     18287|   85039A|           96|
|     18287|    84920|            4|
|     18287|    84584|            6|
|     18287|   84507C|            6|
|     18287|   72351B|           24|
|     18287|   72351A|           24|
|     18287|   72349B|           60|
|     18287|    47422|           24|
|     18287|    47421|           48|
|     18287|    35967|           36|
|     18287|    23445|           20|
|     18287|    23378|           24|
|     18287|    23376|           48|
|     18287|    23310|           36|
|     18287|    23274|           12|
|     18287|    23272|           12|
|     18287|    23269|           36|
+----------+---------+-------------+
only showing top 20 rows



In [41]:
//그룹화 셋을 사용해 동일한 작업 수행
spark.sql("""
SELECT CustomerId, stockCode, sum(Quantity) FROM dfNoNull
GROUP BY customerId, stockCode GROUPING SETS((customerId, stockCode))
ORDER BY CustomerId DESC, stockCode DESC
""").show()

+----------+---------+-------------+
|customerId|stockCode|sum(Quantity)|
+----------+---------+-------------+
|     18287|    85173|           48|
|     18287|   85040A|           48|
|     18287|   85039B|          120|
|     18287|   85039A|           96|
|     18287|    84920|            4|
|     18287|    84584|            6|
|     18287|   84507C|            6|
|     18287|   72351B|           24|
|     18287|   72351A|           24|
|     18287|   72349B|           60|
|     18287|    47422|           24|
|     18287|    47421|           48|
|     18287|    35967|           36|
|     18287|    23445|           20|
|     18287|    23378|           24|
|     18287|    23376|           48|
|     18287|    23310|           36|
|     18287|    23274|           12|
|     18287|    23272|           12|
|     18287|    23269|           36|
+----------+---------+-------------+
only showing top 20 rows



- GROUPING SETS 구문은 SQL에서만 사용할 수 있음
- DataFrame에서 동일한 연산을 수행하려면 rollup 메서드와 cube 메서드 사용

## 7.4.1 롤업
- group-by 스타일의 다양한 연산을 수행할 수 있는 다차원의 집계 기능

In [43]:
//시간(신규 Date 컬럼)과 공간(Country 컬럼)을 축으로 하는 롤업 생성]
//롤업의 결과로 생성된 DataFrame은 모든 날짜의 총합, 날짜별 총합, 날짜별 국가별 총합을 포함

val rolledUpDF = dfNoNull.rollup("Date", "Country").agg(sum("Quantity"))
.selectExpr("Date", "Country", "`sum(Quantity)` as total_quantity")
.orderBy("Date")
rolledUpDF.show()

+----------+--------------+--------------+
|      Date|       Country|total_quantity|
+----------+--------------+--------------+
|      null|          null|       4906888|
|2010-12-01|     Australia|           107|
|2010-12-01|        Norway|          1852|
|2010-12-01|        France|           449|
|2010-12-01|       Germany|           117|
|2010-12-01|          EIRE|           243|
|2010-12-01|   Netherlands|            97|
|2010-12-01|          null|         24032|
|2010-12-01|United Kingdom|         21167|
|2010-12-02|       Germany|           146|
|2010-12-02|          EIRE|             4|
|2010-12-02|          null|         20855|
|2010-12-02|United Kingdom|         20705|
|2010-12-03|      Portugal|            65|
|2010-12-03|         Spain|           400|
|2010-12-03|        Poland|           140|
|2010-12-03|       Belgium|           528|
|2010-12-03|   Switzerland|           110|
|2010-12-03|        France|           239|
|2010-12-03|         Italy|           164|
+----------

rolledUpDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Date: date, Country: string ... 1 more field]


In [44]:
//null 값을 가진 로우에서 전체 날짜의 합계 확인
//롤업된 두 개의 컬럼값이 모두 null인 로우는 두 컬럼에 속한 레코드의 전체 합계를 나타냄

In [45]:
rolledUpDF.where("Country IS NULL").show()

+----------+-------+--------------+
|      Date|Country|total_quantity|
+----------+-------+--------------+
|      null|   null|       4906888|
|2010-12-01|   null|         24032|
|2010-12-02|   null|         20855|
|2010-12-03|   null|         11548|
|2010-12-05|   null|         16394|
|2010-12-06|   null|         16095|
|2010-12-07|   null|         19351|
|2010-12-08|   null|         21275|
|2010-12-09|   null|         16904|
|2010-12-10|   null|         15388|
|2010-12-12|   null|         10561|
|2010-12-13|   null|         15234|
|2010-12-14|   null|         17108|
|2010-12-15|   null|         18169|
|2010-12-16|   null|         29482|
|2010-12-17|   null|         10517|
|2010-12-19|   null|          3735|
|2010-12-20|   null|         12617|
|2010-12-21|   null|         10888|
|2010-12-22|   null|          3053|
+----------+-------+--------------+
only showing top 20 rows



In [46]:
rolledUpDF.where("Date IS NULL").show()

+----+-------+--------------+
|Date|Country|total_quantity|
+----+-------+--------------+
|null|   null|       4906888|
+----+-------+--------------+



## 7.4.2 큐브
- 롤업을 고차원적으로 사용할 수 있게 해줌
- 요소들을 계층적으로 다루는 대신 모든 차원에 대해 동일한 작업을 수행
- 전체 기간에 대해 날짜와 국가별 결과를 얻을 수 있음
    - 전체 날짜와 모든 국가에 대한 합계
    - 모든 국가의 날짜별 합계
    - 날짜별 국가별 합계
    - 전체 날짜의 국가별 합계

In [47]:
dfNoNull.cube("Date", "Country").agg(sum(col("Quantity")))
.select("Date", "Country", "sum(Quantity)").orderBy("Date").show()

+----+--------------------+-------------+
|Date|             Country|sum(Quantity)|
+----+--------------------+-------------+
|null|               Italy|         7999|
|null|      United Kingdom|      4008533|
|null|      Czech Republic|          592|
|null|              Sweden|        35637|
|null|             Finland|        10666|
|null|             Lebanon|          386|
|null|             Iceland|         2458|
|null|             Denmark|         8188|
|null|           Singapore|         5234|
|null|                null|      4906888|
|null|             Germany|       117448|
|null|              Cyprus|         6317|
|null|               Japan|        25218|
|null|             Austria|         4827|
|null|                EIRE|       136329|
|null|        Saudi Arabia|           75|
|null|           Lithuania|          652|
|null|           Australia|        83653|
|null|United Arab Emirates|          982|
|null|              Norway|        19247|
+----+--------------------+-------

## 7.4.3 그룹화 메타데이터
- 큐브와 롤업을 사용하다 보면 집계 수준에 따라 쉽게 필터링하기 위해 집계 수준을 조회하는 경우 발생
- 이때 grouping_id를 사용
- grouping_id는 결과 데이터셋의 집계 수준을 명시하는 컬럼을 제공
    - 3 : 가장 높은 계층의 집계 결과에서 나타남. customerId나 stockCode에 관계 없이 총 수량을 제공
    - 2 : 개별 재고 코드의 모든 집계 결과에서 나타남. customerId에 관계없이 재고 코드별 총 수량을 제공
    - 1 : 구매한 물품에 관계없이 customerId를 기반으로 총 수량을 제공
    - 0 : customerId와 stockCode별 조합에 따라 총 수량을 제공

In [48]:
import org.apache.spark.sql.functions.{grouping_id, sum, expr}

dfNoNull.cube("customerId", "stockCode").agg(grouping_id(), sum("Quantity"))
.orderBy(col("grouping_id()").desc)
.show()

+----------+---------+-------------+-------------+
|customerId|stockCode|grouping_id()|sum(Quantity)|
+----------+---------+-------------+-------------+
|      null|     null|            3|      4906888|
|      null|    21002|            2|           13|
|      null|   84659A|            2|          133|
|      null|    21034|            2|         1872|
|      null|    21619|            2|          900|
|      null|   85049C|            2|         1198|
|      null|    82582|            2|         4271|
|      null|    22643|            2|          185|
|      null|    22642|            2|          253|
|      null|    22717|            2|         1170|
|      null|    21269|            2|           28|
|      null|    21383|            2|          770|
|      null|    22967|            2|         1139|
|      null|    22080|            2|         1172|
|      null|    20713|            2|        10223|
|      null|    85141|            2|          114|
|      null|    22365|         

import org.apache.spark.sql.functions.{grouping_id, sum, expr}


## 7.4.4 피벗
- 로우를 컬럼으로 변환

In [49]:
val pivoted = dfWithDate.groupBy("date").pivot("Country").sum()

pivoted: org.apache.spark.sql.DataFrame = [date: date, Australia_sum(Quantity): bigint ... 113 more fields]


In [50]:
pivoted.where("date > '2011-12-05'").select("date", "`USA_sum(Quantity)`").show()

23/04/23 15:47:02 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
+----------+-----------------+
|      date|USA_sum(Quantity)|
+----------+-----------------+
|2011-12-06|             null|
|2011-12-09|             null|
|2011-12-08|             -196|
|2011-12-07|             null|
+----------+-----------------+



# 7.5 사용자 정의 집계 함수
- 스파크는 입력 데이터의 모든 그룹의 중간 결과를 단일 AggregationBuffer에 저장해 관리
- UDAF를 생성하려면 기본 클래스인 UserDefinedAggregateFunction을 상속받음
- 그리고 다음과 같은 메서드 정의
    - inputSchema
    - bufferSchema
    - dataType
    - deterministic
    - initialize
    - update
    - merge
    - evaluate

In [51]:
import org.apache.spark.sql.expressions.MutableAggregationBuffer
import org.apache.spark.sql.expressions.UserDefinedAggregateFunction
import org.apache.spark.sql.Row
import org.apache.spark.sql.types._

import org.apache.spark.sql.expressions.MutableAggregationBuffer
import org.apache.spark.sql.expressions.UserDefinedAggregateFunction
import org.apache.spark.sql.Row
import org.apache.spark.sql.types._


In [52]:
class BoolAnd extends UserDefinedAggregateFunction{
    def inputSchema: org.apache.spark.sql.types.StructType =
    StructType(StructField("value", BooleanType) :: Nil)
    def bufferSchema: StructType = StructType(
        StructField("result", BooleanType) :: Nil)
    def dataType: DataType = BooleanType
    def deterministic: Boolean = true
    def initialize(buffer: MutableAggregationBuffer): Unit = {
        buffer(0) = true
    }
    def update(buffer: MutableAggregationBuffer, input: Row): Unit = {
        buffer(0) = buffer.getAs[Boolean](0) && input.getAs[Boolean](0)
    }
    def merge(buffer1: MutableAggregationBuffer, buffer2: Row): Unit = {
        buffer1(0) = buffer1.getAs[Boolean](0) && buffer2.getAs[Boolean](0)
    }
    def evaluate(buffer: Row): Any = {
        buffer(0)
    }
}

defined class BoolAnd


In [53]:
val ba = new BoolAnd
spark.udf.register("booland", ba)

import org.apache.spark.sql.functions._

ba: BoolAnd = BoolAnd@44e91381
import org.apache.spark.sql.functions._


In [54]:
spark.range(1)
.selectExpr("explode(array(TRUE, TRUE, TRUE)) as t")
.selectExpr("explode(array(TRUE, FALSE, TRUE)) as f", "t")
.select(ba(col("t")), expr("booland(f)"))
.show()

+----------+----------+
|booland(t)|booland(f)|
+----------+----------+
|      true|     false|
+----------+----------+

